In [1]:
import numpy as np
from szar.counts import ClusterCosmology,Halo_MF
from szar.szproperties import SZ_Cluster_Model
from configparser import SafeConfigParser
from orphics.tools.io import dictFromSection,listFromConfig
import cPickle as pickle
import time

home = '/Users/nab/Repos/SZ_filter/'

!pwd

iniFile = home+'input/pipeline.ini'
expName = 'CCATP-propv2'
gridName = 'grid-owl2'
cal = 'owl2'

Config = SafeConfigParser()
Config.optionxform=str
Config.read(iniFile)

fparams = {}
for (key, val) in Config.items('params'):
    if ',' in val:
        param, step = val.split(',')
        fparams[key] = float(param)
    else:
        fparams[key] = float(val)

bigDataDir = Config.get('general','bigDataDirectory')
clttfile = Config.get('general','clttfile')
constDict = dictFromSection(Config,'constants')
clusterDict = dictFromSection(Config,'cluster_params')
beam = listFromConfig(Config,expName,'beams')
noise = listFromConfig(Config,expName,'noises')
freq = listFromConfig(Config,expName,'freqs')
lknee = listFromConfig(Config,expName,'lknee')[0]
alpha = listFromConfig(Config,expName,'alpha')[0]

calFile = bigDataDir+"lensgrid_grid-"+cal+"_"+cal+".pkl"

version = Config.get('general','version')

mexp_edges, z_edges, lndM = pickle.load(open(calFile,"rb"))

print calFile
print bigDataDir+"szgrid_"+expName+"_"+gridName+ "_v" + version+".pkl"
mgrid,zgrid,siggrid = pickle.load(open(bigDataDir+"szgrid_"+expName+"_"+gridName+ "_v" + version+".pkl",'rb'))

assert np.all(mgrid==mexp_edges)
assert np.all(z_edges==zgrid)

#print lndM

cc = ClusterCosmology(fparams,constDict,clTTFixFile=clttfile)
HMF = Halo_MF(cc,mgrid,zgrid)

qs = listFromConfig(Config,'general','qbins')
qspacing = Config.get('general','qbins_spacing')
if qspacing=="log":
    qbin_edges = np.logspace(np.log10(qs[0]),np.log10(qs[1]),int(qs[2])+1)
elif qspacing=="linear":
    qbin_edges = np.linspace(qs[0],qs[1],int(qs[2])+1)
else:
    raise ValueError
    
SZProp = SZ_Cluster_Model(cc,clusterDict,rms_noises = noise,fwhms=beam,freqs=freq,lknee=lknee,alpha=alpha)

Mwl = 10**HMF.Mexp
z_arr = HMF.zarr
M_arr =  np.outer(HMF.M,np.ones([len(z_arr)]))

#if HMF.sigN is None: HMF.updateSigN(SZProp)
#sigN = HMF.sigN
#np.savetxt(home+'/tests/sigN_arr_save.txt',sigN)

sigN = np.loadtxt(home+'/tests/sigN_arr_save.txt')

#SZProp = SZ_Cluster_Model(cc,clusterDict,rms_noises = noise,fwhms=beam,freqs=freq,lknee=lknee,alpha=alpha)

q_arr = (qbin_edges[1:]+qbin_edges[:-1])/2.

i = 5
kk = 2
jj = 10

start = time.time()
blah_orig = SZProp.P_of_qn(SZProp.lnY,M_arr[:,i],z_arr[i],sigN[:,i],q_arr[kk])
print time.time() - start
blah_mwl = SZProp.Mwl_prob (Mwl[jj],M_arr[:,i],lndM[:,i])

print z_arr.size,q_arr.size,HMF.Mexp.size,Mwl.size

start = time.time()
blah = SZProp.P_of_qn_corr(SZProp.lnY,M_arr[:,i],z_arr[i],sigN[:,i],q_arr,Mwl[jj],lndM[:,i])
print time.time() - start

start = time.time()
test = np.zeros([len(HMF.Mexp),len(q_arr)])
for kk in range(len(q_arr)):
    test[:,kk] = SZProp.P_of_qn(SZProp.lnY,M_arr[:,i],z_arr[i],sigN[:,i],q_arr[kk])
print "orig",time.time() - start

start = time.time()
test2 = SZProp.P_of_qn_arr(SZProp.lnY,M_arr[:,i],z_arr[i],sigN[:,i],q_arr)
print "inner f loop",time.time() - start

print test.shape
print test2.shape

print np.sum(test-test2)

start = time.time()
SZProp.Pfunc_qarr(sigN,HMF.M,z_arr,q_arr)
print time.time() - start
start = time.time()
SZProp.Pfunc_qarr_corr(sigN,HMF.M,z_arr,q_arr,HMF.Mexp,lndM)
print time.time() - start

#print blah_orig * blah_mwl
#print blah

#dN_dmqz_corr = HMF.N_of_mqz_SZ_corr(lndM,qbin_edges,SZProp)                                                                  
#dN_dmqz = HMF.N_of_mqz_SZ(lndM,qbin_edges,SZProp)

/Users/nab/Repos/orphics/orphics/tools/io.py:3: UserWarning: WARNING: This module is deprecated. Most of its contents have moved to orphics.io. If you do not find the function you require there, please raise an issue.
  warnings.warn("WARNING: This module is deprecated. Most of its contents have moved to orphics.io. If you do not find the function you require there, please raise an issue.")
/Users/nab/Repos/orphics/orphics/tools/stats.py:3: UserWarning: WARNING: This module is deprecated. Most of its contents have moved to orphics.stats. If you do not find the function you require there, please raise an issue.
  warnings.warn("WARNING: This module is deprecated. Most of its contents have moved to orphics.stats. If you do not find the function you require there, please raise an issue.")
/Users/nab/Repos/orphics/orphics/analysis/flatMaps.py:3: UserWarning: WARNING: This module is deprecated. Most of its contents have moved to orphics.maps. If you do not find the function you require ther

/Users/nab/Repos/SZ_filter
/Users/nab/Desktop/Projects/SO_forecasts/lensgrid_grid-owl2_owl2.pkl
/Users/nab/Desktop/Projects/SO_forecasts/szgrid_CCATP-propv2_grid-owl2_v0.7.pkl
0.00389099121094
20 64 222 222
0.470403909683
orig 0.212644100189
inner f loop 0.0509779453278
(222, 64)
(222, 64)
0.0
1.01853990555
2157.76593709


/Users/nab/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:18: DeprecationWarning: The SafeConfigParser class has been renamed to ConfigParser in Python 3.2. This alias will be removed in future versions. Use ConfigParser directly instead.
/Users/nab/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:20: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.


In [ ]:
print (blah - (blah_orig * blah_mwl)) / blah

In [ ]:
from szar.szproperties import gaussianMat2D
#from szar.szproperties import gaussianMat2D_old

NN = 222
LL = 130
rho = 0.5

randarr3 = np.random.random([2,NN,LL])
randarr = np.random.random([NN,LL])
randarr2 = np.random.random([NN,LL])
randl = np.arange(LL)*0.1
randm = np.random.random([NN])

ans = np.zeros([NN])
ans3 = np.zeros([NN,LL])
ans4 = np.zeros([NN,LL])

print randarr.shape, randarr2.shape,randl.shape,(np.diff(randl)).shape, (randarr*randarr2).shape
start = time.time()
for ii in range(NN):
    ans3[ii,:] = gaussianMat2D(randarr3[:,ii,:],1.,randm[ii],rho)
    #ans[ii] = np.trapz(randarr[ii,:]*randarr2[ii,:],randl,np.diff(randl))
#print ans
print time.time() - start

#start = time.time()
#for ii in range(NN):
#    ans4[ii,:] = gaussianMat2D_old(randarr3[:,ii,:],1.,randm[ii],rho)
    #ans[ii] = np.trapz(randarr[ii,:]*randarr2[ii,:],randl,np.diff(randl))
#print ans
#print time.time() - start

start = time.time()
for ii in range(NN):
    #ans3[ii,:] = gaussianMat2D(randarr[:,ii,:],1.,randm[ii],rho)
    ans[ii] = np.trapz(randarr[ii,:]*randarr2[ii,:],randl,np.diff(randl))
#print ans
print time.time() - start

start = time.time()
ans2 = np.trapz(randarr*randarr2,randl,np.diff(randl),axis=1)
print time.time() - start
print np.sum(ans2-ans)
print np.sum(ans3-ans4)
#ans2 = np.apply_along_axis(gaussianMat2D,1,randarr,1.,randm,rho)